# Basic Analytics

### Connect To MongoDB

In [2]:
## Import MongoClient
# from pymongo import MongoClient

## Connect To DB
# Remove When Committing
# connection_string = '' 
# client = MongoClient(connection_string)
# db = client['test']

In [ ]:
## Testing
# print(list(db['vehicles'].find()))

## Analytics Dashboard

In [3]:
## General Libraries
import json
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns

### Job Analytics

#### Job Type Distribution

In [4]:
# General Labels
job_types = ['ELECTRICIAN', 'VENTILATION', 'PLUMBER', 'HANDYMAN', 'AIRCON']

In [5]:
# Past 1 Month
total_job_frequency = [21, 6, 3, 19, 11]

data_job_type_dist = {
    'labels': job_types,
    'data': total_job_frequency
}

# Save data to JSON file
with open('../public/analytics/job-type-dist-one-month-total.json', 'w') as file:
    json.dump(data_job_type_dist, file, indent=4)

average_job_frequency = total_job_frequency

data_job_type_dist = {
    'labels': job_types,
    'data': average_job_frequency
}

# Save data to JSON file
with open('../public/analytics/job-type-dist-one-month-average.json', 'w') as file:
    json.dump(data_job_type_dist, file, indent=4)   

In [6]:
# Past 3 Month
total_job_frequency = [56, 17, 10, 61, 29]

data_job_type_dist = {
    'labels': job_types,
    'data': total_job_frequency
}

# Save data to JSON file
with open('../public/analytics/job-type-dist-three-month-total.json', 'w') as file:
    json.dump(data_job_type_dist, file, indent=4)  

average_job_frequency = [x / 3 for x in total_job_frequency]

data_job_type_dist = {
    'labels': job_types,
    'data': average_job_frequency
}

# Save data to JSON file
with open('../public/analytics/job-type-dist-three-month-average.json', 'w') as file:
    json.dump(data_job_type_dist, file, indent=4) 

In [7]:
# Past 6 Month
total_job_frequency = [112, 33, 19, 106, 55]

data_job_type_dist = {
    'labels': job_types,
    'data': total_job_frequency
}

# Save data to JSON file
with open('../public/analytics/job-type-dist-six-month-total.json', 'w') as file:
    json.dump(data_job_type_dist, file, indent=4)

average_job_frequency = [x / 6 for x in total_job_frequency]

data_job_type_dist = {
    'labels': job_types,
    'data': average_job_frequency
}

# Save data to JSON file
with open('../public/analytics/job-type-dist-six-month-average.json', 'w') as file:
    json.dump(data_job_type_dist, file, indent=4) 

In [8]:
# Past 12 Month
total_job_frequency = [222, 61, 40, 205, 99]

data_job_type_dist = {
    'labels': job_types,
    'data': total_job_frequency
}

# Save data to JSON file
with open('../public/analytics/job-type-dist-twelve-month-total.json', 'w') as file:
    json.dump(data_job_type_dist, file, indent=4)

average_job_frequency = [x / 12 for x in total_job_frequency]

data_job_type_dist = {
    'labels': job_types,
    'data': average_job_frequency
}

# Save data to JSON file
with open('../public/analytics/job-type-dist-twelve-month-average.json', 'w') as file:
    json.dump(data_job_type_dist, file, indent=4) 

#### Job Duration Distribution

In [9]:
job_duration = [21, 48, 57, 19, 181]

data_job_duration_dist = {
    'labels': job_types,
    'data': job_duration
}

# Save data to JSON file
with open('../public/analytics/job-duration-dist.json', 'w') as file:
    json.dump(data_job_duration_dist, file, indent=4) 